# Análisis de usuarios más activos en una conversaciones de Twittter.

## Éste notebook es parte del material para el taller de  **análisis de narrativas emergentes, discurso de odio y desinformación en red** desarollado por **Heurística** dentro del projecto DataPolitik.

Permite analizar más activo en una discusión de Twitter y obtener sus scores de [Botometer](https://botometer.osome.iu.edu/) si se dispone de acceso a la [API](https://botometer.osome.iu.edu/api).

## Cómo obtener los datos

El tipo de análisis que vamos a desarrollar en este notebook será útil para los casos en que queremos:

- análizar el alcance de una campaña
- monitorear la emergencia o el estancamiento de narrativas

En cada uno de estos casos empezamos con una pregunta más o menos abstracta, el siguiente paso es entender cómo traducir ésta pregunta a una consulta de búsqueda en la API de Twitter.

Se pueden dar dos casos:

 - el tema que queremos estudiar está identificado con un hashtag
 - el tema que queremos estudiar es más amplio y/o difuso.
 
En el primer caso nuestra consulta será simplemente el hashtag en cuestión. 
En el segundo, tendremos que encontrar un conjunto de palabras claves (y de combinaciones entre ellas) que nos permitan capturar una parte representativa de los tweets y retweets sobre el tema. 

Existen varias herramientas para ejecutar consultas a la API de Twitter, en éste caso asumimos que los hemos obtenidos usando el comando [search](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/#counts) de [twarc2](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/).

# Configuración del notebook

La celda siguiente contiene todos los *import* de las librerias necesarias para la sesión y la configuración de estilo para las graficas. 

In [1]:
import pandas as pd
import json
import botometer
import numpy as np
from twarc import ensure_flattened
from io import TextIOWrapper

/home/emanuele/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


En la celda siguiente hay que ingresar las claves para la API de Botometer si se dispone de acceso. 

In [2]:
rapidapi_key = ""
twitter_app_auth = {
    'consumer_key': '',
    'consumer_secret': '',
    'access_token': '',
    'access_token_secret': '',
  }
botometer_api_url = 'https://botometer-pro.p.mashape.com'
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

La celda siguiente prepara el formato de datos. 

In [3]:
class most_active_user(object):
    def __init__(self, name,tweets,tot_tweets):
        self.username=name
        self.profile_image=tweets.iloc[0]['author.profile_image_url']
        self.id=tweets.iloc[0]['author.id']
        self.created_at=tweets.iloc[0]['author.created_at']
        self.description=tweets.iloc[0]['author.description']
        self.tot_activity=tweets['text'].count()/float(tot_tweets)*100
        self.rt=self.rt_c(tweets)
        self.t=int(tot_tweets*self.tot_activity/100-self.rt)
        #self.botscore=self.botometerscore(tweets.iloc[0]['author.id']) #DECOMENTAR ESTA LÍNEA SI SE QUIERE USAR BOTOMETER
    def rt_c(self,tweets):
        count=0
        for row in tweets.iterrows():
            if type(row[1]['referenced_tweets'])==list and len(row[1]['referenced_tweets'])==1:
                
                
                if not pd.isna(row[1]['referenced_tweets']):
                    if row[1]['referenced_tweets'][0]['type']=='retweeted':
                        count+=1
        return count
    def botometerscore(self,uid):
        result = bom.check_account(uid)
        return result['cap']['universal']
    def to_dict(self):
        return {
            'username': self.username,
            'profile mage':self.profile_image,
            'twitter id': self.id,
            'created_at': self.created_at,
            'description': self.description,
            'activity share(%)': round(self.tot_activity,2),
            'retweets': self.rt,
            'tweets': self.t
            #'botometer score': self.botscore #DECOMENTAR ESTA LÍNEA SI SE QUIERE USAR BOTOMETER
        }

# Carga y preparación de los datos

En primer lugar, cargamos el conjunto de datos. 


In [4]:
results=[]
with open('../../racismo_datapolitik/Badaoui.jsonl', 'r') as json_file:
    json_list = list(json_file)

for line in json_list:
    data = json.loads(line)
    for tweet in ensure_flattened(data):
        results.append(tweet)

In [5]:
datadf=pd.json_normalize(results)

In [6]:
activity_ordered_users=datadf.groupby('author.username')['id'].nunique().sort_values(ascending=False).reset_index(name='count')

Hemos obtenidos una tabla con todos los usuarios ordenados por el numero de tuits y retuits emitidos por cada uno.

In [7]:
activity_ordered_users.head(20)

,author.username,count
0,helios_f,170
1,Reusantifa,113
2,_Maria_Dantas_,96
3,MohamedSaidBad2,91
4,MusulmanaDDHH,86
5,UCFRcat,84
6,Ali7Mohamed,82
7,vic_ga101,78
8,VilaWeb,75
9,silviasaiz_1714,74


Enriquecemos la tabla con más datos de los usuarios, restringiendola a los primeros N más activos. 

In [8]:
maus=[]
N = 20
tot_tweets=datadf['id'].count()
for row1 in activity_ordered_users.head(N).iterrows():
    rslt_df = datadf[datadf['author.username'] == row1[1]['author.username']]
    maus.append(most_active_user(rslt_df.iloc[0]['author.username'],rslt_df,tot_tweets))
mausdf=pd.DataFrame.from_records([s.to_dict() for s in maus])

Calculamos el porcentage de trafico generado por los N primeros usuarios más activos, segun el resultado podemos decidir ampliar o reducir N. 

In [9]:
mausdf['activity share(%)'].sum()

3.65

In [10]:
mausdf.head()

,username,profile mage,twitter id,created_at,description,activity share(%),retweets,tweets
0,helios_f,https://pbs.twimg.com/profile_images/161929877...,1194866907779915777,2019-11-14T06:37:41.000Z,"✍🏽 en @ctxt_es, @ElSaltoDiario y en otros espa...",0.42,138,32
1,Reusantifa,https://pbs.twimg.com/profile_images/100043217...,1000431187238760449,2018-05-26T17:39:16.000Z,Xarxa antifeixista plural i diversa d'informac...,0.28,113,0
2,_Maria_Dantas_,https://pbs.twimg.com/profile_images/120222752...,87042194,2009-11-02T22:10:54.000Z,"Sergipana, barcelonina, jurista, activista soc...",0.24,89,7
3,MohamedSaidBad2,https://pbs.twimg.com/profile_images/161038819...,1047388047095021568,2018-10-03T07:29:04.000Z,Musulmà i Activista social🕋🌍.\nNo només ets re...,0.23,85,6
4,MusulmanaDDHH,https://pbs.twimg.com/profile_images/113885036...,1047861318924034050,2018-10-04T14:49:41.000Z,La Asociación Musulmana por los #DerechosHuman...,0.21,86,0


Salvamos los datos en un formato que permite su visualización através esta plantilla de [fluorish](https://public.flourish.studio/visualisation/12665613/)

In [11]:
mausdf.index = np.arange(1, len(mausdf)+1)
mausdf.to_csv('../BADAOUImaus.csv')